In [182]:
from ortools.linear_solver import pywraplp
from weasyprint import HTML, CSS
import tempfile
import os

In [183]:
import json 

with open('data.json') as json_file:
    data = json.load(json_file)

In [184]:
data.keys()

dict_keys(['instructors', 'lessons', 'classrooms'])

In [185]:
def get_instructor_by_name(instructors, name):
    for instructor in instructors:
        if instructor["name"] == name:
            return instructor
    return None

### With Group Information

In [186]:
from ortools.linear_solver import pywraplp


def get_instructor_by_name(instructors, name):
    for instructor in instructors:
        if instructor["name"] == name:
            return instructor
    return None


def solve_schedule(data):
    solver = pywraplp.Solver.CreateSolver("SCIP")
    if not solver:
        return

    instructors = data["instructors"]
    lessons = data["lessons"]
    classrooms = data["classrooms"]

    # Remove instructors with empty lesson lists
    instructors = [instructor for instructor in instructors if instructor["lessons"]]

    # Remove duplicate lessons with same group numbers
    unique_lessons = {}
    for lesson in lessons:
        key = (lesson["name"], lesson["grade"], lesson["group"])
        if key not in unique_lessons:
            unique_lessons[key] = lesson
    lessons = list(unique_lessons.values())

    time_slots = list(range(8, 17))  # 8:00 AM to 5:00 PM
    days_of_week = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
    grades = [1, 2, 3, 4]
    lunch_start = 12
    lunch_end = 13

    # Print debug information
    print(f"Total number of lessons: {len(lessons)}")
    print(f"Total number of instructors: {len(instructors)}")
    print(f"Total number of classrooms: {len(classrooms)}")
    print(f"Time slots: {time_slots}")
    print(
        "Available time slots per day:",
        len(
            [
                t
                for t in time_slots
                if not (t < lunch_start and t + 3 > lunch_start) and not (t >= lunch_start and t < lunch_end)
            ]
        ),
    )
    print("\nStarting solver...\n")

    # Decision variables
    x = {}
    for lesson in lessons:
        for instructor in lesson["instructors"]:
            instructor_data = get_instructor_by_name(instructors, instructor)
            if not instructor_data:
                print(f"Warning: Instructor {instructor} not found in instructors list")
                continue

            # Only create variables for instructor's preferred days
            instructor_days = instructor_data["preferred_days"]
            for day in instructor_days:
                # Only create variables for valid start times that don't overlap with lunch
                for time in range(8, 17 - lesson["duration"] + 1):
                    # Skip if lesson would overlap with lunch break
                    if time < lunch_start and time + lesson["duration"] > lunch_start:
                        continue
                    if time >= lunch_start and time < lunch_end:
                        continue

                    if lesson["type"] in ["FaceToFace", "Hybrid"]:
                        for classroom in classrooms:
                            x[(lesson["name"], lesson["group"], instructor, classroom, day, time)] = solver.BoolVar(
                                f'x_{lesson["name"]}_{lesson["group"]}_{instructor}_{classroom}_{day}_{time}'
                            )
                    else:  # Online classes
                        x[(lesson["name"], lesson["group"], instructor, None, day, time)] = solver.BoolVar(
                            f'x_{lesson["name"]}_{lesson["group"]}_{instructor}_None_{day}_{time}'
                        )

    print(f"Number of variables created: {len(x)}")

    # Constraint 1: Each lesson-group combination must be scheduled exactly once
    for lesson in lessons:
        constraint_expr = []
        for instructor in lesson["instructors"]:
            instructor_data = get_instructor_by_name(instructors, instructor)
            if not instructor_data:
                continue

            instructor_days = instructor_data["preferred_days"]
            for day in instructor_days:
                for time in range(8, 17 - lesson["duration"] + 1):
                    if time < lunch_start and time + lesson["duration"] > lunch_start:
                        continue
                    if time >= lunch_start and time < lunch_end:
                        continue

                    if lesson["type"] in ["FaceToFace", "Hybrid"]:
                        for classroom in classrooms:
                            if (lesson["name"], lesson["group"], instructor, classroom, day, time) in x:
                                constraint_expr.append(
                                    x[(lesson["name"], lesson["group"], instructor, classroom, day, time)]
                                )
                    else:
                        if (lesson["name"], lesson["group"], instructor, None, day, time) in x:
                            constraint_expr.append(x[(lesson["name"], lesson["group"], instructor, None, day, time)])

        if constraint_expr:
            solver.Add(sum(constraint_expr) == 1)

    # Constraint 2: Instructor cannot teach overlapping classes
    for instructor in instructors:
        for day in days_of_week:
            for time in time_slots:
                constraint_expr = []
                for lesson in lessons:
                    if instructor["name"] in lesson["instructors"]:
                        # Check all possible start times that would overlap with current time slot
                        for start_time in range(
                            max(8, time - lesson["duration"] + 1), min(time + 1, 17 - lesson["duration"] + 1)
                        ):
                            if start_time < lunch_start and start_time + lesson["duration"] > lunch_start:
                                continue
                            if start_time >= lunch_start and start_time < lunch_end:
                                continue

                            if lesson["type"] in ["FaceToFace", "Hybrid"]:
                                for classroom in classrooms:
                                    if (
                                        lesson["name"],
                                        lesson["group"],
                                        instructor["name"],
                                        classroom,
                                        day,
                                        start_time,
                                    ) in x:
                                        constraint_expr.append(
                                            x[
                                                (
                                                    lesson["name"],
                                                    lesson["group"],
                                                    instructor["name"],
                                                    classroom,
                                                    day,
                                                    start_time,
                                                )
                                            ]
                                        )
                            else:
                                if (lesson["name"], lesson["group"], instructor["name"], None, day, start_time) in x:
                                    constraint_expr.append(
                                        x[(lesson["name"], lesson["group"], instructor["name"], None, day, start_time)]
                                    )
                if constraint_expr:
                    solver.Add(sum(constraint_expr) <= 1)

    # Constraint 3: Classroom cannot have overlapping classes
    for classroom in classrooms:
        for day in days_of_week:
            for time in time_slots:
                constraint_expr = []
                for lesson in lessons:
                    if lesson["type"] in ["FaceToFace", "Hybrid"]:
                        for start_time in range(
                            max(8, time - lesson["duration"] + 1), min(time + 1, 17 - lesson["duration"] + 1)
                        ):
                            if start_time < lunch_start and start_time + lesson["duration"] > lunch_start:
                                continue
                            if start_time >= lunch_start and start_time < lunch_end:
                                continue

                            for instructor in lesson["instructors"]:
                                if (lesson["name"], lesson["group"], instructor, classroom, day, start_time) in x:
                                    constraint_expr.append(
                                        x[(lesson["name"], lesson["group"], instructor, classroom, day, start_time)]
                                    )
                if constraint_expr:
                    solver.Add(sum(constraint_expr) <= 1)

    # Constraint 4: No overlapping mandatory classes (whether different or same lessons)
    # The only exception is different groups of the same lesson can overlap
    for grade in grades:
        for day in days_of_week:
            for time in time_slots:
                # Get all mandatory lessons for this grade
                mandatory_lessons = []
                for lesson in lessons:
                    if lesson["grade"] == grade and lesson["obligation"] == "mandatory":
                        mandatory_lessons.append(lesson)

                # Create constraint for all mandatory lessons
                for lesson1 in mandatory_lessons:
                    # Check all possible start times that would overlap with current time slot
                    for start_time in range(
                        max(8, time - lesson1["duration"] + 1), min(time + 1, 17 - lesson1["duration"] + 1)
                    ):
                        if start_time < lunch_start and start_time + lesson1["duration"] > lunch_start:
                            continue
                        if start_time >= lunch_start and start_time < lunch_end:
                            continue

                        # Get all variables for this lesson at this time
                        lesson1_expr = []
                        for instructor in lesson1["instructors"]:
                            if lesson1["type"] in ["FaceToFace", "Hybrid"]:
                                for classroom in classrooms:
                                    if (
                                        lesson1["name"],
                                        lesson1["group"],
                                        instructor,
                                        classroom,
                                        day,
                                        start_time,
                                    ) in x:
                                        lesson1_expr.append(
                                            x[
                                                (
                                                    lesson1["name"],
                                                    lesson1["group"],
                                                    instructor,
                                                    classroom,
                                                    day,
                                                    start_time,
                                                )
                                            ]
                                        )
                            else:  # Online classes
                                if (lesson1["name"], lesson1["group"], instructor, None, day, start_time) in x:
                                    lesson1_expr.append(
                                        x[(lesson1["name"], lesson1["group"], instructor, None, day, start_time)]
                                    )

                        # For each other mandatory lesson that's not a different group of the same lesson
                        for lesson2 in mandatory_lessons:
                            if (
                                lesson1["name"] != lesson2["name"]
                            ):  # Skip if they're the same lesson (different groups allowed to overlap)
                                for start_time2 in range(
                                    max(8, time - lesson2["duration"] + 1), min(time + 1, 17 - lesson2["duration"] + 1)
                                ):
                                    if start_time2 < lunch_start and start_time2 + lesson2["duration"] > lunch_start:
                                        continue
                                    if start_time2 >= lunch_start and start_time2 < lunch_end:
                                        continue

                                    lesson2_expr = []
                                    for instructor in lesson2["instructors"]:
                                        if lesson2["type"] in ["FaceToFace", "Hybrid"]:
                                            for classroom in classrooms:
                                                if (
                                                    lesson2["name"],
                                                    lesson2["group"],
                                                    instructor,
                                                    classroom,
                                                    day,
                                                    start_time2,
                                                ) in x:
                                                    lesson2_expr.append(
                                                        x[
                                                            (
                                                                lesson2["name"],
                                                                lesson2["group"],
                                                                instructor,
                                                                classroom,
                                                                day,
                                                                start_time2,
                                                            )
                                                        ]
                                                    )
                                        else:  # Online classes
                                            if (
                                                lesson2["name"],
                                                lesson2["group"],
                                                instructor,
                                                None,
                                                day,
                                                start_time2,
                                            ) in x:
                                                lesson2_expr.append(
                                                    x[
                                                        (
                                                            lesson2["name"],
                                                            lesson2["group"],
                                                            instructor,
                                                            None,
                                                            day,
                                                            start_time2,
                                                        )
                                                    ]
                                                )

                                    # If both lessons have variables at these times, add constraint
                                    if lesson1_expr and lesson2_expr:
                                        solver.Add(sum(lesson1_expr) + sum(lesson2_expr) <= 1)

    # Constraint 5: Different groups of same lesson can overlap in time but need different classrooms
    for lesson_name in set(lesson["name"] for lesson in lessons):
        lesson_groups = [lesson for lesson in lessons if lesson["name"] == lesson_name]
        if len(lesson_groups) > 1 and lesson_groups[0]["type"] in ["FaceToFace", "Hybrid"]:
            for day in days_of_week:
                for time in time_slots:
                    for classroom in classrooms:
                        constraint_expr = []
                        for lesson in lesson_groups:
                            for instructor in lesson["instructors"]:
                                if (lesson["name"], lesson["group"], instructor, classroom, day, time) in x:
                                    constraint_expr.append(
                                        x[(lesson["name"], lesson["group"], instructor, classroom, day, time)]
                                    )
                        if constraint_expr:
                            # Same classroom can't be used by different groups at same time
                            # But different groups can overlap if using different classrooms
                            solver.Add(sum(constraint_expr) <= 1)

    # Constraint 6: Each instructor can only teach one group of a lesson
    for lesson_name in set(lesson["name"] for lesson in lessons):
        lesson_groups = [lesson for lesson in lessons if lesson["name"] == lesson_name]
        if len(lesson_groups) > 1:  # If the lesson has multiple groups
            # For each instructor that can teach this lesson
            for instructor in set(instr for lesson in lesson_groups for instr in lesson["instructors"]):
                instructor_group_vars = []
                for lesson in lesson_groups:
                    # Collect all variables for this instructor across all days and times
                    for day in days_of_week:
                        for time in range(8, 17 - lesson["duration"] + 1):
                            if time < lunch_start and time + lesson["duration"] > lunch_start:
                                continue
                            if time >= lunch_start and time < lunch_end:
                                continue

                            if lesson["type"] in ["FaceToFace", "Hybrid"]:
                                for classroom in classrooms:
                                    if (lesson["name"], lesson["group"], instructor, classroom, day, time) in x:
                                        instructor_group_vars.append(
                                            x[(lesson["name"], lesson["group"], instructor, classroom, day, time)]
                                        )
                            else:  # Online classes
                                if (lesson["name"], lesson["group"], instructor, None, day, time) in x:
                                    instructor_group_vars.append(
                                        x[(lesson["name"], lesson["group"], instructor, None, day, time)]
                                    )

                if instructor_group_vars:
                    solver.Add(sum(instructor_group_vars) <= 1)  # Instructor can only teach one group

    # Solve
    solver.SetTimeLimit(300000)  # 5 minutes time limit
    status = solver.Solve()

    # Print results
    if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
        print("\nSolution found:\n")
        for lesson in lessons:
            for instructor in lesson["instructors"]:
                for day in days_of_week:
                    for time in range(8, 17 - lesson["duration"] + 1):
                        if time < lunch_start and time + lesson["duration"] > lunch_start:
                            continue
                        if time >= lunch_start and time < lunch_end:
                            continue

                        if lesson["type"] in ["FaceToFace", "Hybrid"]:
                            for classroom in classrooms:
                                if (lesson["name"], lesson["group"], instructor, classroom, day, time) in x:
                                    if (
                                        x[
                                            (lesson["name"], lesson["group"], instructor, classroom, day, time)
                                        ].solution_value()
                                        == 1
                                    ):
                                        print(
                                            f"Lesson: {lesson['name']} (Grade {lesson['grade']}, Group {lesson['group']}) - Instructor: {instructor}"
                                        )
                                        print(f"Day: {day}, Time: {time}:00 - {time + lesson['duration']}:00")
                                        print(f"Type: {lesson['type']}, Classroom: {classroom}")
                                        print(f"Obligation: {lesson['obligation']}\n")
                        else:
                            if (lesson["name"], lesson["group"], instructor, None, day, time) in x:
                                if (
                                    x[(lesson["name"], lesson["group"], instructor, None, day, time)].solution_value()
                                    == 1
                                ):
                                    print(
                                        f"Lesson: {lesson['name']} (Grade {lesson['grade']}, Group {lesson['group']}) - Instructor: {instructor}"
                                    )
                                    print(f"Day: {day}, Time: {time}:00 - {time + lesson['duration']}:00")
                                    print(f"Type: {lesson['type']} (No classroom needed)")
                                    print(f"Obligation: {lesson['obligation']}\n")
    else:
        print("No solution found within the time limit.")


In [187]:
solve_schedule(data)

Total number of lessons: 41
Total number of instructors: 31
Total number of classrooms: 9
Time slots: [8, 9, 10, 11, 12, 13, 14, 15, 16]
Available time slots per day: 6

Starting solver...

Number of variables created: 12080

Solution found:

Lesson: MAT1071_MATEMTIK1 (Grade 1, Group 1) - Instructor: Pinar_ALbayrak2
Day: Wednesday, Time: 8:00 - 11:00
Type: FaceToFace, Classroom: D-106
Obligation: mandatory

Lesson: MAT1071_MATEMTIK1 (Grade 1, Group 2) - Instructor: Pinar_ALbayrak1
Day: Wednesday, Time: 8:00 - 11:00
Type: FaceToFace, Classroom: D-109
Obligation: mandatory

Lesson: ATA1031_INKULAP_TARIHI (Grade 1, Group 1) - Instructor: Kerem_Karaosmanoglu
Day: Friday, Time: 8:00 - 11:00
Type: Online (No classroom needed)
Obligation: mandatory

Lesson: FIZ1001_Fizik_1 (Grade 1, Group 1) - Instructor: HARIS_KULOSMANOVIC1
Day: Tuesday, Time: 8:00 - 11:00
Type: FaceToFace, Classroom: D-107
Obligation: mandatory

Lesson: FIZ1001_Fizik_1 (Grade 1, Group 2) - Instructor: HARIS_KULOSMANOVIC2
Da

In [255]:
output= """


Total number of lessons: 41
Total number of instructors: 31
Total number of classrooms: 9
Time slots: [8, 9, 10, 11, 12, 13, 14, 15, 16]
Available time slots per day: 6

Starting solver...

Number of variables created: 12080

Solution found:

Lesson: MAT1071_MATEMTIK1 (Grade 1, Group 1) - Instructor: Pinar_ALbayrak2
Day: Wednesday, Time: 8:00 - 11:00
Type: FaceToFace, Classroom: D-106
Obligation: mandatory

Lesson: MAT1071_MATEMTIK1 (Grade 1, Group 2) - Instructor: Pinar_ALbayrak1
Day: Wednesday, Time: 8:00 - 11:00
Type: FaceToFace, Classroom: D-109
Obligation: mandatory

Lesson: ATA1031_INKULAP_TARIHI (Grade 1, Group 1) - Instructor: Kerem_Karaosmanoglu
Day: Friday, Time: 8:00 - 11:00
Type: Online (No classroom needed)
Obligation: mandatory

Lesson: FIZ1001_Fizik_1 (Grade 1, Group 1) - Instructor: HARIS_KULOSMANOVIC1
Day: Tuesday, Time: 8:00 - 11:00
Type: FaceToFace, Classroom: D-107
Obligation: mandatory

Lesson: FIZ1001_Fizik_1 (Grade 1, Group 2) - Instructor: HARIS_KULOSMANOVIC2
Day: Tuesday, Time: 8:00 - 11:00
Type: FaceToFace, Classroom: D-110
Obligation: mandatory

Lesson: MAT1320_Lineer_Cebir (Grade 1, Group 1) - Instructor: Kayahan_Sanli2
Day: Tuesday, Time: 13:00 - 16:00
Type: FaceToFace, Classroom: D-109
Obligation: mandatory

Lesson: MAT1320_Lineer_Cebir (Grade 1, Group 2) - Instructor: Kayahan_Sanli1
Day: Tuesday, Time: 13:00 - 16:00
Type: FaceToFace, Classroom: D-106
Obligation: mandatory

Lesson: MDB1031_Ileri_Ingilizce_I (Grade 1, Group 1) - Instructor: Ece_Aksan1
Day: Monday, Time: 13:00 - 16:00
Type: FaceToFace, Classroom: D-107
Obligation: mandatory

Lesson: MDB1031_Ileri_Ingilizce_I (Grade 1, Group 2) - Instructor: Ece_Aksan2
Day: Monday, Time: 13:00 - 16:00
Type: FaceToFace, Classroom: D-110
Obligation: mandatory

Lesson: BLM1011_Bilgisayar_Bilimlerine_Giris (Grade 1, Group 1) - Instructor: M_Amac_Guvensan
Day: Monday, Time: 8:00 - 11:00
Type: FaceToFace, Classroom: D-107
Obligation: mandatory

Lesson: BLM1011_Bilgisayar_Bilimlerine_Giris (Grade 1, Group 2) - Instructor: Goksel_Biricik
Day: Wednesday, Time: 13:00 - 16:00
Type: FaceToFace, Classroom: D-106
Obligation: mandatory

Lesson: BLM1011_Bilgisayar_Bilimlerine_Giris (Grade 1, Group 3) - Instructor: H_Irem_Turkmen
Day: Thursday, Time: 13:00 - 16:00
Type: FaceToFace, Classroom: D-107
Obligation: mandatory

Lesson: BLM1991_Is_Sag_Guv_1 (Grade 1, Group 1) - Instructor: Adem_Altay
Day: Thursday, Time: 8:00 - 11:00
Type: FaceToFace, Classroom: D-106
Obligation: mandatory

Lesson: BLM2012_Nesneye_Yonelik_Prog (Grade 2, Group 1) - Instructor: Y_Emre_Selcuk
Day: Monday, Time: 13:00 - 16:00
Type: FaceToFace, Classroom: D-106
Obligation: mandatory

Lesson: BLM2012_Nesneye_Yonelik_Prog (Grade 2, Group 2) - Instructor: M_Siddik_Aktas
Day: Monday, Time: 13:00 - 16:00
Type: FaceToFace, Classroom: D-109
Obligation: mandatory

Lesson: BLM2012_Nesneye_Yonelik_Prog (Grade 2, Group 3) - Instructor: Furkan_Cakmak
Day: Thursday, Time: 13:00 - 16:00
Type: FaceToFace, Classroom: D-106
Obligation: mandatory

Lesson: BLM2611_Lojik_Devreler (Grade 2, Group 1) - Instructor: Gokhan_Bilgin
Day: Monday, Time: 10:00 - 12:00
Type: FaceToFace, Classroom: D-109
Obligation: mandatory

Lesson: BLM2611_Lojik_Devreler (Grade 2, Group 2) - Instructor: H_Osman_Ilhan
Day: Monday, Time: 10:00 - 12:00
Type: FaceToFace, Classroom: D-106
Obligation: mandatory

Lesson: BLM2521_Ayrik_Matematik (Grade 2, Group 1) - Instructor: Ahmet_Elbir
Day: Monday, Time: 8:00 - 10:00
Type: FaceToFace, Classroom: D-106
Obligation: mandatory

Lesson: BLM2521_Ayrik_Matematik (Grade 2, Group 2) - Instructor: Banu_Diri
Day: Monday, Time: 8:00 - 10:00
Type: FaceToFace, Classroom: D-109
Obligation: mandatory

Lesson: BLM2011_Ist_ve_Olasilik_Hes (Grade 2, Group 1) - Instructor: Ayse_Dalgali
Day: Wednesday, Time: 8:00 - 11:00
Type: FaceToFace, Classroom: D-107
Obligation: mandatory

Lesson: BLM2011_Ist_ve_Olasilik_Hes (Grade 2, Group 2) - Instructor: Oguz_Altun
Day: Wednesday, Time: 8:00 - 11:00
Type: FaceToFace, Classroom: D-108
Obligation: mandatory

Lesson: BLM2642_Bilg_Muh_Icin_Diff_Denk (Grade 2, Group 1) - Instructor: M_Fatih_Amasyali
Day: Tuesday, Time: 8:00 - 11:00
Type: FaceToFace, Classroom: D-106
Obligation: mandatory

Lesson: BLM2642_Bilg_Muh_Icin_Diff_Denk (Grade 2, Group 2) - Instructor: Ayse_Dalgali
Day: Tuesday, Time: 8:00 - 11:00
Type: FaceToFace, Classroom: D-109
Obligation: mandatory

Lesson: BLM3042_Seminer_ve_Meslek_Etigi (Grade 2, Group 1) - Instructor: Y_Emre_Selcuk
Day: Tuesday, Time: 13:00 - 16:00
Type: FaceToFace, Classroom: D-107
Obligation: mandatory

Lesson: BLM3042_Seminer_ve_Meslek_Etigi (Grade 2, Group 2) - Instructor: Goksel_Biricik
Day: Tuesday, Time: 13:00 - 16:00
Type: FaceToFace, Classroom: D-110
Obligation: mandatory

Lesson: BLM3021_Algoritma_Analizi (Grade 3, Group 1) - Instructor: M_Elif_Karsligil
Day: Monday, Time: 13:00 - 16:00
Type: FaceToFace, Classroom: D-108
Obligation: mandatory

Lesson: BLM3021_Algoritma_Analizi (Grade 3, Group 2) - Instructor: M_Amac_Guvensan
Day: Monday, Time: 13:00 - 16:00
Type: FaceToFace, Classroom: D-111
Obligation: mandatory

Lesson: BLM3011_Isletim_Sistemleri (Grade 3, Group 1) - Instructor: Ahmet_Elbir
Day: Tuesday, Time: 8:00 - 11:00
Type: FaceToFace, Classroom: D-108
Obligation: mandatory

Lesson: BLM3110_BMoK_1 (Grade 3, Group 1) - Instructor: Emine_Dumlu_Demircioglu
Day: Wednesday, Time: 9:00 - 12:00
Type: FaceToFace, Classroom: D-110
Obligation: elective

Lesson: BLM3041_Veritabani_Yonetimi (Grade 3, Group 1) - Instructor: M_Utku_Kalay
Day: Tuesday, Time: 13:00 - 16:00
Type: FaceToFace, Classroom: D-108
Obligation: mandatory

Lesson: BLM3061_Mikroisl_Sist_ve_Assmb (Grade 3, Group 1) - Instructor: ERKAN_USLU
Day: Monday, Time: 8:00 - 11:00
Type: FaceToFace, Classroom: D-108
Obligation: mandatory

Lesson: BLM3061_Mikroisl_Sist_ve_Assmb (Grade 3, Group 2) - Instructor: Furkan_Cakmak
Day: Thursday, Time: 8:00 - 11:00
Type: FaceToFace, Classroom: D-107
Obligation: mandatory

Lesson: BLM3740_Yoneylem_Arastirmasi (Grade 4, Group 1) - Instructor: Ayse_Buharli_Olcasoy
Day: Monday, Time: 14:00 - 17:00
Type: FaceToFace, Classroom: DB-011
Obligation: elective

Lesson: BLM3120_Bil_Er_Ar_Mot (Grade 3, Group 1) - Instructor: M_Siddik_Aktas
Day: Monday, Time: 9:00 - 12:00
Type: FaceToFace, Classroom: DB-011
Obligation: elective

Lesson: BLM3730_Blokzincir_Temelleri (Grade 3, Group 1) - Instructor: Oguz_Altun
Day: Monday, Time: 9:00 - 12:00
Type: FaceToFace, Classroom: D-111
Obligation: elective

Lesson: BLM3780_Ver_Tab_Sis_Gerc (Grade 3, Group 1) - Instructor: M_Utku_Kalay
Day: Monday, Time: 9:00 - 12:00
Type: FaceToFace, Classroom: D-010
Obligation: elective

Lesson: BLM4011_Bilisim_Sis_Guvenligi (Grade 4, Group 1) - Instructor: Ali_Gokhan_Yavuz
Day: Friday, Time: 9:00 - 12:00
Type: Online
Obligation: elective

Lesson: BLM4710_Yonetim_Bilgi_Sis (Grade 4, Group 1) - Instructor: Oya_Kalipsiz
Day: Monday, Time: 9:00 - 12:00
Type: FaceToFace, Classroom: D-012
Obligation: elective

Lesson: BLM4770_Yaz_Kal_Test_Sur (Grade 4, Group 1) - Instructor: Oya_Kalipsiz
Day: Monday, Time: 14:00 - 17:00
Type: FaceToFace, Classroom: D-010
Obligation: elective

Lesson: BLM4021_Gomulu_Sistemler (Grade 4, Group 1) - Instructor: Ali_Can_Karaca
Day: Monday, Time: 9:00 - 12:00
Type: FaceToFace, Classroom: D-110
Obligation: elective




 """

In [256]:
from weasyprint import HTML, CSS
import tempfile
import os


def generate_schedule_pdf(solution_data, output_path="schedule.pdf"):
    css_content = """
        @page {
            size: A3 portrait;
            margin: 0.5cm;
        }
        
        body { 
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 0;
        }
        
        .schedule-title {
            text-align: center;
            margin: 5px 0 10px 0;
            padding: 5px;
            background-color: #FFFF00;
            border: 1px solid #808000;
        }
        
        .university {
            font-size: 16pt;
            font-weight: bold;
            margin-bottom: 2px;
        }
        
        .department {
            font-size: 14pt;
            font-weight: bold;
            margin-bottom: 2px;
        }
        
        .subtitle {
            font-size: 12pt;
            font-weight: bold;
        }
        
        table {
            width: 100%;
            border-collapse: collapse;
            font-size: 8pt;
            table-layout: fixed;
        }
        
        th, td {
            border: 1px solid black;
            padding: 1px;
            text-align: center;
            vertical-align: top;
            height: 18px;
            overflow: hidden;
        }
        
        td:empty {
            height: 8px !important;
            padding: 0;
            border-top: 1px dotted #ccc;
            border-bottom: 1px dotted #ccc;
        }
        
        .time-cell {
            width: 60px;
            background-color: #f0f0f0;
            font-size: 6pt;
            padding: 1px;
            white-space: nowrap;
            text-align: center;
        }
        
        .time-range {
            margin-top: 1px;
            font-weight: normal;
            color: #333;
            text-align: center;
            padding: 1px 0;
        }
        
        .day-monday { background-color: #FFE4E1; }
        .day-tuesday { background-color: #E0FFFF; }
        .day-wednesday { background-color: #E6E6FA; }
        .day-thursday { background-color: #F0FFF0; }
        .day-friday { background-color: #FFF0F5; }
        
        .grade-header {
            background-color: #4a4a4a;
            color: white;
            font-weight: bold;
            font-size: 9pt;
            height: 25px;
            padding: 2px;
        }
        
        .course-cell { 
            min-height: 18px; 
            padding: 1px;
            margin: 0;
            position: relative;
        }
        
        .face-to-face { 
            background-color: #98FB98;
            border-left: 3px solid #228B22;
        }
        .online { 
            background-color: #B3E0FF;
            border-left: 3px solid #0066CC;
        }
        .hybrid { 
            background-color: #FFA07A;
            border-left: 3px solid #FF4500;
        }
        
        .course-code {
            font-weight: bold;
            font-size: 7pt;
            padding: 1px;
            margin: 0;
            white-space: nowrap;
            overflow: hidden;
            text-overflow: ellipsis;
            text-align: center;
            background: rgba(255,255,255,0.4);
            border-radius: 2px;
        }

        .course-code.mandatory {
            color: #800000;  /* Dark red for mandatory courses */
        }

        .course-code.elective {
            color: #006400;  /* Dark green for elective courses */
        }
        
        .course-info {
            font-size: 7pt;
            color: #444;
            text-align: center;
            padding: 1px;
            margin: 1px 0;
            background: rgba(255,255,255,0.5);
            border-radius: 2px;
            font-weight: bold;
        }
        
        .groups-container {
            display: flex;
            flex-direction: row;
            gap: 1px;
            margin-top: 1px;
            background: rgba(255,255,255,0.2);
            padding: 1px;
            border-radius: 2px;
        }
        
        .group-info {
            flex: 1;
            font-size: 6pt;
            padding: 1px;
            background: rgba(255,255,255,0.3);
            border-radius: 2px;
            border-left: 1px dotted rgba(0,0,0,0.2);
        }
        
        .group-info:first-child {
            border-left: none;
        }
        
        .group-header {
            font-weight: bold;
            color: #444;
            border-bottom: 1px dotted #999;
            margin-bottom: 2px;
            font-size: 6pt;
            text-align: center;
            background: rgba(255,255,255,0.4);
            border-radius: 2px;
            padding: 2px;
        }
        
        .course-instructor {
            font-style: italic;
            font-size: 6pt;
            margin: 1px 0;
            line-height: 1.2;
            text-align: center;
        }
        
        .course-details { 
            font-size: 6pt;
            color: #444;
            line-height: 1.2;
            text-align: center;
            margin-top: 2px;
        }
        
        .electives-container {
            display: flex;
            flex-direction: column;
            gap: 1px;
            margin-top: 1px;
            border-top: 2px dashed #666;
            padding-top: 1px;
        }
        
        .elective-course {
            width: 100%;
            background: rgba(255,255,255,0.2);
            border-radius: 2px;
            padding: 1px;
            position: relative;
        }
        
        .elective-course:not(:last-child) {
            border-bottom: 1px dashed #666;
            padding-bottom: 1px;
            margin-bottom: 1px;
        }
        
        .day-label {
            font-weight: bold;
            display: block;
            margin-bottom: 0;
            color: #444;
            border-bottom: 1px solid #999;
            padding-bottom: 0;
            background-color: rgba(0,0,0,0.1);
            font-size: 6pt;
            line-height: 1.1;
        }
    """

    days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
    grades = [1, 2, 3, 4]
    times = list(range(8, 18))

    # Create schedule grid with support for multiple groups
    schedule_grid = {day: {grade: {time: [] for time in times} for grade in grades} for day in days}

    # Populate schedule grid
    for lesson in solution_data:
        day = lesson["day"]
        grade = lesson["grade"]
        start_time = lesson["start_time"]
        duration = lesson["duration"]

        # Add lesson only to the start time slot
        schedule_grid[day][grade][start_time].append(lesson)

    # Generate HTML
    html_content = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <meta charset="UTF-8">
    </head>
    <body>
        <div class="schedule-title">
            <div class="university">Yıldız Technical University</div>
            <div class="department">Department of Computer Engineering</div>
            <div class="subtitle">Course Schedule</div>
        </div>
        <table>
            <tr>
                <th class="time-cell"></th>
    """

    for grade in grades:
        html_content += f'<th class="grade-header">{grade}. Grade</th>'
    html_content += "</tr>"

    def generate_lesson_cell_content(lessons):
        if not lessons:
            return ""

        # First, separate mandatory and elective courses
        mandatory_courses = {}
        elective_courses = {}

        for lesson in lessons:
            course_name = lesson["name"]
            course_data = {
                "lessons": [],
                "duration": lesson["duration"],
                "type": lesson["type"],
                "is_elective": lesson.get("is_elective", False),
            }

            if lesson.get("is_elective", False):
                if course_name not in elective_courses:
                    elective_courses[course_name] = course_data
                elective_courses[course_name]["lessons"].append(lesson)
            else:
                if course_name not in mandatory_courses:
                    mandatory_courses[course_name] = course_data
                mandatory_courses[course_name]["lessons"].append(lesson)

        content = ""

        # Handle mandatory courses
        for course_name, course_data in mandatory_courses.items():
            type_class = course_data["type"].lower().replace(" ", "-")
            content += f'<div class="course-cell {type_class}">'
            content += f'<div class="course-code mandatory">{course_name} ({course_data["duration"]} hours)</div>'
            content += f'<div class="course-info">Mandatory - {course_data["type"]}</div>'

            # Groups container
            content += '<div class="groups-container">'
            for lesson in sorted(course_data["lessons"], key=lambda x: x.get("group", 1)):
                content += f"""
                    <div class="group-info">
                        <div class="group-header">Group {lesson.get("group", 1)}</div>
                        <div class="course-instructor">{lesson["instructor"]}</div>
                        {f'<div class="course-details">{lesson["classroom"]}</div>' if lesson.get("classroom") and lesson["type"] != "Online" else ""}
                    </div>
                """
            content += "</div></div>"

        # Handle elective courses
        if elective_courses:
            content += '<div class="electives-container">'
            for course_name, course_data in elective_courses.items():
                type_class = course_data["type"].lower().replace(" ", "-")
                content += f'<div class="elective-course {type_class}">'
                content += f'<div class="course-code elective">{course_name} ({course_data["duration"]} hours)</div>'
                content += f'<div class="course-info">Elective - {course_data["type"]}</div>'

                # Groups container for this elective course
                content += '<div class="groups-container">'
                for lesson in sorted(course_data["lessons"], key=lambda x: x.get("group", 1)):
                    content += f"""
                        <div class="group-info">
                            <div class="group-header">Group {lesson.get("group", 1)}</div>
                            <div class="course-instructor">{lesson["instructor"]}</div>
                            {f'<div class="course-details">{lesson["classroom"]}</div>' if lesson.get("classroom") and lesson["type"] != "Online" else ""}
                        </div>
                    """
                content += "</div></div>"
            content += "</div>"

        return content

    # Generate rows for each day and time
    current_day = None
    for day in days:
        for time in times:
            row_html = "<tr>"

            # Generate time cell with proper formatting
            time_cell_content = f"{time:02d}:00 - {(time+1):02d}:00"

            if current_day != day:
                row_html += f"""<td class="time-cell day-{day.lower()}">
                    <span class="day-label">{day}</span>
                    <div class="time-range">{time_cell_content}</div></td>"""
                current_day = day
            else:
                row_html += f'<td class="time-cell day-{day.lower()}">{time_cell_content}</td>'

            # Add cells for each grade
            for grade in grades:
                lessons_at_time = schedule_grid[day][grade][time]
                cell_added = False

                # Check if this time slot is part of a previous lesson's duration
                is_continuation = False
                for prev_time in range(max(8, time - 5), time):
                    prev_lessons = schedule_grid[day][grade][prev_time]
                    for prev_lesson in prev_lessons:
                        if prev_time + prev_lesson["duration"] > time:
                            is_continuation = True
                            break
                    if is_continuation:
                        break

                if not is_continuation:
                    if not lessons_at_time:
                        row_html += "<td></td>"
                    else:
                        # Calculate rowspan based on the maximum duration of lessons starting at this time
                        max_duration = max(lesson["duration"] for lesson in lessons_at_time)
                        row_html += f'<td rowspan="{max_duration}" class="course-cell {lessons_at_time[0]["type"].lower().replace(" ", "-")}">'
                        row_html += generate_lesson_cell_content(lessons_at_time)
                        row_html += "</td>"
                    cell_added = True

            row_html += "</tr>"
            html_content += row_html

    html_content += """
        </table>
    </body>
    </html>
    """

    # Create temporary files for HTML and CSS
    with tempfile.NamedTemporaryFile(suffix=".html", mode="w", encoding="utf-8", delete=False) as html_file:
        html_file.write(html_content)
        html_path = html_file.name

    with tempfile.NamedTemporaryFile(suffix=".css", mode="w", encoding="utf-8", delete=False) as css_file:
        css_file.write(css_content)
        css_path = css_file.name

    try:
        HTML(html_path).write_pdf(output_path, stylesheets=[CSS(filename=css_path)])
    finally:
        os.unlink(html_path)
        os.unlink(css_path)


def format_solver_solution(solver_output):
    """
    Convert solver output to the format needed by generate_schedule_pdf.
    """
    formatted_data = []
    current_lesson = {}

    # Clean and split the output into lines
    lines = [line.strip() for line in solver_output.strip().splitlines() if line.strip()]

    for line in lines:
        if line.startswith("Lesson:"):
            if current_lesson:
                formatted_data.append(current_lesson)
            current_lesson = {}
            # Parse lesson line with group information
            lesson_parts = line.split(" - ")
            lesson_info = lesson_parts[0].replace("Lesson: ", "")
            name_grade_group = lesson_info.split(" (Grade ")
            name = name_grade_group[0].strip()
            grade_group = name_grade_group[1].replace(")", "").split(", Group ")
            current_lesson["name"] = name
            current_lesson["grade"] = int(grade_group[0])
            current_lesson["group"] = int(grade_group[1]) if len(grade_group) > 1 else 1
            current_lesson["instructor"] = lesson_parts[1].replace("Instructor: ", "").strip()

        elif line.startswith("Day:"):
            day_time = line.split(", ")
            current_lesson["day"] = day_time[0].replace("Day: ", "").strip()
            time_range = day_time[1].replace("Time: ", "").split(" - ")
            current_lesson["start_time"] = int(time_range[0].split(":")[0])
            current_lesson["duration"] = int(time_range[1].split(":")[0]) - current_lesson["start_time"]

        elif line.startswith("Type:"):
            type_info = line.split(", ")
            current_lesson["type"] = type_info[0].replace("Type: ", "").split(" ")[0].strip()
            if len(type_info) > 1:
                current_lesson["classroom"] = type_info[1].replace("Classroom: ", "").strip()

        elif line.startswith("Obligation:"):
            obligation = line.replace("Obligation:", "").strip()
            current_lesson["is_elective"] = obligation.lower() != "mandatory"

    # Append the last lesson if it exists
    if current_lesson:
        formatted_data.append(current_lesson)

    return formatted_data


In [257]:
formatted_data = format_solver_solution(output)

In [258]:
generate_schedule_pdf(formatted_data, "ascheduleNEW.pdf")

In [254]:
formatted_data

[{'name': 'MAT1071_MATEMTIK1',
  'grade': 1,
  'group': 1,
  'instructor': 'Pinar_ALbayrak2',
  'day': 'Wednesday',
  'start_time': 8,
  'duration': 3,
  'type': 'FaceToFace',
  'classroom': 'D-106',
  'is_elective': False},
 {'name': 'MAT1071_MATEMTIK1',
  'grade': 1,
  'group': 2,
  'instructor': 'Pinar_ALbayrak1',
  'day': 'Wednesday',
  'start_time': 8,
  'duration': 3,
  'type': 'FaceToFace',
  'classroom': 'D-109',
  'is_elective': False},
 {'name': 'ATA1031_INKULAP_TARIHI',
  'grade': 1,
  'group': 1,
  'instructor': 'Kerem_Karaosmanoglu',
  'day': 'Friday',
  'start_time': 8,
  'duration': 3,
  'type': 'Online',
  'is_elective': False},
 {'name': 'FIZ1001_Fizik_1',
  'grade': 1,
  'group': 1,
  'instructor': 'HARIS_KULOSMANOVIC1',
  'day': 'Tuesday',
  'start_time': 8,
  'duration': 3,
  'type': 'FaceToFace',
  'classroom': 'D-107',
  'is_elective': False},
 {'name': 'FIZ1001_Fizik_1',
  'grade': 1,
  'group': 2,
  'instructor': 'HARIS_KULOSMANOVIC2',
  'day': 'Tuesday',
  'sta